# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from IPython.display import Markdown
import os
from time import time

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
)

from api_key_store import ApiKeyStore
from helper import convert_to_filename

# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
s = ApiKeyStore()

In [3]:
os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'   # gpt-4 was default, throws rate-limit error 

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful support representative in your team",
	backstory="""
		You work at crewAI (https://crewai.com) and are now working on providing 
		support to {customer}, a super important customer  for your company.
		You need to make sure that you provide the best support!
		Make sure to provide full complete answers, and make no assumptions.
	""",
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the best support quality assurance in your team",
	backstory="""
		You work at crewAI (https://crewai.com) and are now working with your team 
		on a request from {customer} ensuring that the support representative is 
		providing the best support possible.\n
		You need to make sure that the support representative is providing full complete answers, 
        and make no assumptions.
	""",
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [7]:
inquiry_resolution = Task(
    description="""
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    """.replace('"',''),
    expected_output="""
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    """.replace('"',''),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [8]:
quality_assurance_review = Task(
    description="""
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    """.replace('"',''),
    expected_output="""
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    """.replace('"',''),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [10]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": """
            I need help with setting up a Crew and kicking it off, 
            specifically how can I add memory to my crew? 
            Can you provide guidance?
        """
}
ts_start = time()
result = crew.kickoff(inputs=inputs)
ts_stop = time()

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: 
        DeepLearningAI just reached out with a super important ask:

	    
            I need help with setting up a Crew and kicking it off, 
            specifically how can I add memory to my crew? 
            Can you provide guidance?
        


        Andrew Ng from DeepLearningAI is the one that reached out. 
		Make sure to use everything you know 
        to provide the best support possible.
		You must strive to provide a complete 
        and accurate response to the customer's inquiry.
    


> Entering new CrewAgentExecutor chain...
I need to provide the best support to DeepLearningAI, so I should use the tools available to gather all the necessary information before crafting a detailed response.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crew

In [11]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")



Elapsed time: 26.19 sec


- Display the final result as Markdown.

In [12]:
Markdown(result)

Hello Andrew Ng from DeepLearningAI,

Thank you for reaching out with your inquiry about setting up a Crew and adding memory to it. I understand the importance of this task for your business, and I am here to provide you with comprehensive guidance.

To set up a Crew and kick it off, you can follow the steps outlined in our documentation. I recommend referring to the specific guide on "Creating a Crew and kicking it off" available on our website at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/. This detailed resource will walk you through the process step-by-step, ensuring that you have a smooth setup experience.

Now, regarding adding memory to your Crew, it's essential to consider the specific requirements and capabilities of your Crew members. Memory in this context could refer to the storage capacity or the ability of your Crew to retain information and learn. To enhance memory in your Crew, you can:

1. Encourage regular communication and collaboration among Crew members to promote knowledge sharing and retention.
2. Provide training sessions, workshops, or resources that can help improve the memory and skills of your Crew members.
3. Utilize AI tools and technologies to enhance data storage and processing capabilities within your Crew.

By implementing these strategies, you can effectively add memory to your Crew and optimize its performance for successful outcomes.

If you have any further questions or need additional assistance, please feel free to reach out. I am dedicated to providing you with the best support possible to help you achieve your goals.

Best regards,

[Your Name]
Senior Support Representative
crewAI